<a href="https://www.kaggle.com/code/jihunshim/07-huggingface-whole-processings?scriptVersionId=221049986" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# # 1. 환경 변수 설정
# API 키를 환경 변수로 설정하는 코드입니다.
# OPENAI_API_KEY: OpenAI의 API를 사용할 때 필요하지만, 이 코드에서는 사용되지 않음.
# HF_TOKEN: Hugging Face에서 모델과 데이터셋을 다운로드할 때 필요한 인증 토큰.

import os
os.environ["OPENAI_API_KEY"] = ''
os.environ["HF_TOKEN"] = ''

## 데이터셋 관련

In [ ]:
# 2. 데이터셋 로드 및 전처리

from datasets import load_dataset

In [ ]:
# klue/ynat 데이터셋(한국어 뉴스 주제 분류 데이터)을 훈련(train)과 검증(validation) 데이터셋으로 불러옴.
# 이 데이터는 뉴스 제목(title)과 해당 뉴스의 주제 라벨(label)로 구성됨

# https://huggingface.co/datasets/klue/klue/viewer/ynat
klue_ynat_train = load_dataset('klue','ynat',split='train')
klue_ynat_validation = load_dataset('klue','ynat',split='validation')

In [ ]:
# 이 데이터는 뉴스 제목과 주제(0~6 사이의 숫자로 표현된 라벨)를 포함합니다.
# 👉 라벨(0~6)은 뉴스 카테고리를 의미하며, klue_ynat_train.features['label'].names를 확인하면 각 숫자가 어떤 카테고리인지 알 수 있습니다.

# type(klue_ynat_train)
klue_ynat_train[0]

In [ ]:
vars(klue_ynat_train).keys()

In [ ]:
# vars(klue_ynat_train)['_info'] # 데이터셋이 가진 여러 정보
klue_ynat_train.features['label'].names # 설명 카테고리

In [ ]:
# (1) 불필요한 컬럼 삭제
# 뉴스 분류에는 title(제목)과 label(라벨)만 필요하므로 불필요한 정보(guid, url, date)를 삭제.

klue_ynat_train_data = klue_ynat_train.remove_columns(['guid','url','date'])
klue_ynat_validation_data = klue_ynat_validation.remove_columns(['guid','url','date'])

In [ ]:
klue_ynat_train_data, klue_ynat_validation_data

In [ ]:
# type(klue_ynat_train.features['label'])
klue_ynat_train.features['label']

In [ ]:
# (2) 라벨 숫자를 실제 이름으로 변환
# 6이라는 숫자가 어떤 뉴스 카테고리를 의미하는지 확인하는 코드입니다.
# 예를 들어 6이 스포츠라면, int2str(6) = "스포츠"가 됩니다.

# type(klue_ynat_train.features['label'])
# klue_ynat_train.features['label'].int2str(6)
klue_ynat_train_data.features['label'].int2str(6)

### 데이터셋 분할

In [ ]:
# (3) 훈련 데이터셋을 더 나누기 (훈련/테스트)

# klue_ynat_train_data_split = klue_ynat_train_data.train_test_split(test_size=0.3, shuffle=True, seed=24) # 원래는 0.3정도가 좋음
klue_ynat_train_data_split = klue_ynat_train_data.train_test_split(test_size=0.8, shuffle=True, seed=24)

# klue_ynat_train_data_split['test']
klue_ynat_train_data_split

In [ ]:
klue_ynat_train_data = klue_ynat_train_data_split['train']

## 모델 관련

In [ ]:
# 3. 모델 불러오기
# klue/roberta-base 모델을 사용하여 7개 클래스(0~6)의 뉴스 주제 분류 모델을 만듦.
# num_labels=7: 출력 뉴스를 7개의 카테고리로 분류하도록 모델을 설정.

from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = 'klue/roberta-base'
out_features = len(klue_ynat_train.features['label'].names)

In [ ]:
# AutoModelForSequenceClassification.from_pretrained(model_id) # out_feature 랜덤하게 부여됨
model_ynat = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=7)

In [ ]:
model_ynat.state_dict()

In [ ]:
# 4. 토크나이저 설정 및 데이터 토큰화
# AutoTokenizer.from_pretrained(model_id): 문장을 토큰으로 변환하는 함수(BERT 기반 토큰화).
# tokenize_function: 뉴스 제목(title)을 토큰화하는 함수.
# map(tokenize_function, batched=True): 데이터셋을 한 번에 여러 개(batch)씩 변환.

tokenizer = AutoTokenizer.from_pretrained(model_id)
def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

In [ ]:
train_dataset = klue_ynat_train_data.map(tokenize_function, batched=True)
validation_dataset = klue_ynat_validation_data.map(tokenize_function, batched=True)

In [ ]:
# type(train_dataset)
# train_dataset
train_dataset[0].keys()

In [ ]:
# 5. 훈련 설정 및 평가 함수 정의
# num_train_epochs=1: 1번만 데이터셋을 학습(에포크 수).
# per_device_train_batch_size=8: 훈련 배치 크기(한 번에 8개 샘플을 처리).
# per_device_eval_batch_size=8: 검증 배치 크기.
# eval_strategy='epoch': 매 에포크마다 검증 진행.

from transformers import Trainer, TrainingArguments
repo_id = 'roberta-base-ynat-classification'

training_args = TrainingArguments(output_dir=repo_id
                 , num_train_epochs=1
                 , per_device_train_batch_size=8
                 , per_device_eval_batch_size=8
                 , eval_strategy='epoch'
                 , learning_rate=0.001
                 , push_to_hub=False
                 , logging_steps=1
                 , report_to="none"  # WandB, TensorBoard 등 모두 비활성화
                                 )

In [ ]:
# 모델 평가 기준 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
# 모델의 예측 아이디와 문자열 레이블을 연결할 데이터를 모델 config에 저장
id2label = {i: label for i, label in enumerate(train_dataset.features['label'].names)}
label2id = {label: i for i, label in id2label.items()}
model_ynat.config.id2label = id2label
model_ynat.config.label2id = label2id

In [ ]:
trainer = Trainer(model = model_ynat
       , args = training_args
       , train_dataset = train_dataset
       , eval_dataset = validation_dataset
       , tokenizer = tokenizer
       , compute_metrics = compute_metrics)

In [ ]:
import numpy as np
# 파인튜닝 시작
trainer.train()

## 모델 평가

In [ ]:
# 정확도율 확인
trainer.evaluate(validation_dataset)

## 모델 서비스

In [ ]:
# # 모델 업로드 to huggingface
# from huggingface_hub import login
# login()

# trainer.push_to_hub(repo_id)

In [ ]:
from transformers import pipeline
repo_id = 'shim-jh/results_ynat'

model_pipeline = pipeline('text-classification', model=repo_id)

In [ ]:
model_pipeline(train_dataset[4:10]['title'])